# 了解onnx结构
## 1.0 执行，导出一个简单网络的onnx

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.onnx
import os

class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.conv = nn.Conv2d(1, 1, 3, padding=1)
        self.relu = nn.ReLU()
        self.conv.weight.data.fill_(1)
        self.conv.bias.data.fill_(0)
    
    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        return x

# 这个包对应opset11的导出代码，如果想修改导出的细节，可以在这里修改代码
# import torch.onnx.symbolic_opset11
print("对应opset文件夹代码在这里：", os.path.dirname(torch.onnx.__file__))

model = Model()
dummy = torch.zeros(1, 1, 3, 3)
torch.onnx.export(
    model, 

    # 这里的args，是指输入给model的参数，需要传递tuple，因此用括号
    # 杨秀勇：注意这里必须是元祖。  dummy是一个和输入数据形状一样的即可。
    (dummy,), 

    # 储存的文件路径
    "demo.onnx", 

    # 打印详细信息
    verbose=True, 

    # 为输入和输出节点指定名称，方便后面查看或者操作
    input_names=["image"], 
    output_names=["output"], 

    # 这里的opset，指，各类算子以何种方式导出，对应于symbolic_opset11
    opset_version=11, 

    # 表示他有batch、height、width3个维度是动态的，在onnx中给其赋值为-1
    # 通常，我们只设置batch为动态，其他的避免动态
    # 数字代表维度 N C H W,给字母或-1，表示该维度是动态输入的。0: "batch"表示0维度是动态的
    dynamic_axes={
        "image": {0: "batch", 2: "height", 3: "width"}, 
        "output": {0: "batch", 2: "height", 3: "width"},
    }
)

print("Done.!")

对应opset文件夹代码在这里： /usr/local/lib/python3.8/dist-packages/torch/onnx
graph(%image : Float(*, 1, *, *, strides=[9, 9, 3, 1], requires_grad=0, device=cpu),
      %conv.weight : Float(1, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=1, device=cpu),
      %conv.bias : Float(1, strides=[1], requires_grad=1, device=cpu)):
  %3 : Float(*, 1, *, *, strides=[9, 9, 3, 1], requires_grad=1, device=cpu) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[3, 3], pads=[1, 1, 1, 1], strides=[1, 1]](%image, %conv.weight, %conv.bias) # /usr/local/lib/python3.8/dist-packages/torch/nn/modules/conv.py:442:0
  %output : Float(*, 1, *, *, strides=[9, 9, 3, 1], requires_grad=1, device=cpu) = onnx::Relu(%3) # /usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1299:0
  return (%output)

Done.!


## 2.0 读取onnx

In [4]:
import onnx
import onnx.helper as helper
import numpy as np

model = onnx.load("demo.onnx")

#打印信息
print("==============node信息")
# print(helper.printable_graph(model.graph))
print(model)

conv_weight = model.graph.initializer[0]
conv_bias = model.graph.initializer[1]

# 数据是以protobuf的格式存储的，因此当中的数值会以bytes的类型保存，通过np.frombuffer方法还原成类型为float32的ndarray
print(f"===================={conv_weight.name}==========================")
print(conv_weight.name, np.frombuffer(conv_weight.raw_data, dtype=np.float32))
# conv.weight [0. 1. 2. 3. 4. 5. 6. 7. 8.]

print(f"===================={conv_bias.name}==========================")
print(conv_bias.name, np.frombuffer(conv_bias.raw_data, dtype=np.float32))
# conv.bias [0.]

==============node信息
ir_version: 7
opset_import {
  version: 11
}
producer_name: "pytorch"
producer_version: "1.10"
graph {
  node {
    input: "image"
    input: "conv.weight"
    input: "conv.bias"
    output: "3"
    name: "Conv_0"
    op_type: "Conv"
    attribute {
      name: "dilations"
      type: INTS
      ints: 1
      ints: 1
    }
    attribute {
      name: "group"
      type: INT
      i: 1
    }
    attribute {
      name: "kernel_shape"
      type: INTS
      ints: 3
      ints: 3
    }
    attribute {
      name: "pads"
      type: INTS
      ints: 1
      ints: 1
      ints: 1
      ints: 1
    }
    attribute {
      name: "strides"
      type: INTS
      ints: 1
      ints: 1
    }
    doc_string: "/usr/local/lib/python3.8/dist-packages/torch/nn/modules/conv.py(442): _conv_forward\n/usr/local/lib/python3.8/dist-packages/torch/nn/modules/conv.py(446): forward\n/usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py(1090): _slow_forward\n/usr/local/lib/pyth

## 3.0 编辑onnx

In [ ]:
import onnx
import onnx.helper as helper
import numpy as np

model = onnx.load("demo.onnx")

# 可以取出权重
conv_weight = model.graph.initializer[0]
conv_bias = model.graph.initializer[1]
# 修改权
conv_weight.raw_data = np.arange(9, dtype=np.float32).tobytes()

# 修改权重后储存
onnx.save_model(model, "demo.change.onnx")
print("Done.!")

## 4.0 最底层创建onnx
- ModelProto:当加载了一个onnx后，会获得一个ModelProto。
- GraphProto: 包含了四个repeated数组(可以用来存放N个相同类型的内容，key值为数字序列类型.)。这四个数组分别是node(NodeProto类型)，input(ValueInfoProto类型)，output(ValueInfoProto类型)和initializer(TensorProto类型)；
- NodeProto: 存node，放了模型中所有的计算节点,语法结构如下：
- ValueInfoProto: 存input，放了模型的输入节点。存output，放了模型中所有的输出节点；
- TensorProto: 存initializer，放了模型的所有权重参数
- AttributeProto:每个计算节点中还包含了一个AttributeProto数组，用来描述该节点的属性，比如Conv节点或者说卷积层的属性包含group，pad，strides等等；

In [ ]:
import onnx # pip install onnx>=1.10.2
import onnx.helper as helper # helper的作用就是帮我们更好的创建节点
import numpy as np

# https://github.com/onnx/onnx/blob/v1.2.1/onnx/onnx-ml.proto
'''
make_node(op_type: str, inputs: Sequence[str], outputs: Sequence[str], name: Union[str, NoneType] = None, doc_string: Union[str, NoneType] = None, domain: Union[str, NoneType] = None, **kwargs: Any) -> onnx.onnx_ml_pb2.NodeProto
    Construct a NodeProto.
    
    Arguments:
        op_type (string): The name of the operator to construct
        inputs (list of string): list of input names
        outputs (list of string): list of output names
        name (string, default None): optional unique identifier for NodeProto
        doc_string (string, default None): optional documentation string for NodeProto
        domain (string, default None): optional domain for NodeProto.
            If it's None, we will just use default domain (which is empty)
        **kwargs (dict): the attributes of the node.  The acceptable values
            are documented in :func:`make_attribute`.
        kwargs到底是啥？。不同的op_type这个参数不同，要去官网看
            以Conv为例子。https://onnx.ai/onnx/operators/onnx__Conv.html#conv
                中，看Attributes。kwargs指的就是这里
            以ReLU为例子。官网没有Attributes，所有就没有多余的值
    Returns:
        NodeProto
'''

nodes = [ # !!!注意，下方make_node的顺序和 具体api不一样。name的位置换了下。不重要
    helper.make_node( # 创建NodeProto
        name="Conv_0",   # 节点名字，不要和op_type搞混了
        op_type="Conv",  # 节点的算子类型, 比如'Conv'、'Relu'、'Add'这类，详细可以参考onnx给出的算子列表
        inputs=["image", "conv.weight", "conv.bias"],  # 各个输入的名字，结点的输入包含：输入和算子的权重。必有输入X和权重W，偏置B可以作为可选。
        outputs=["3"],  
        pads=[1, 1, 1, 1], # 其他字符串为节点的属性，attributes在官网被明确的给出了，标注了default的属性具备默认值。
        group=1,
        dilations=[1, 1],
        kernel_shape=[3, 3],
        strides=[1, 1]
    ),
    helper.make_node(
        name="ReLU_1",
        op_type="Relu",
        inputs=["3"],
        outputs=["output"]
    )
]

initializer = [
    helper.make_tensor( # 创建TensorProto
        name="conv.weight",
        data_type=helper.TensorProto.DataType.FLOAT,
        dims=[1, 1, 3, 3],
        vals=np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], dtype=np.float32).tobytes(),
        raw=True
    ),
    helper.make_tensor(
        name="conv.bias",
        data_type=helper.TensorProto.DataType.FLOAT,
        dims=[1],
        vals=np.array([0.0], dtype=np.float32).tobytes(),
        raw=True
    )
]

inputs = [
    helper.make_value_info( #创建ValueInfoProto
        name="image",
        type_proto=helper.make_tensor_type_proto(
            elem_type=helper.TensorProto.DataType.FLOAT,
            shape=["batch", 1, 3, 3]
        )
    )
]

outputs = [
    helper.make_value_info(
        name="output",
        type_proto=helper.make_tensor_type_proto(
            elem_type=helper.TensorProto.DataType.FLOAT,
            shape=["batch", 1, 3, 3]
        )
    )
]

graph = helper.make_graph(
    name="mymodel",
    inputs=inputs,
    outputs=outputs,
    nodes=nodes,
    initializer=initializer
)

# 如果名字不是ai.onnx，netron解析就不是太一样了
opset = [
    helper.make_operatorsetid("ai.onnx", 11)
]

# producer主要是保持和pytorch一致
model = helper.make_model(graph, opset_imports=opset, producer_name="pytorch", producer_version="1.9")
onnx.save_model(model, "my.onnx")

print(model)
print("Done.!")